In [61]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [62]:
# Notes from Phil
# euphotic depths are in light/martini800_v9ae_z01_PAR0_monmean.nc,
# while the marine heat wave index is in bottom/martini800_v9ae_MHWI_s0.nc
# and has only 3 time steps since it is a yearly index.
# The other variables should all have 36 time steps.
# The DOC variable is called ‘B1_DOC’.

In [63]:
def closest_inx(lat1, lon1, lat2, lon2):
    # lat1, lon1 - 2D rho arrays
    # lat2, lon2 - target coordinates
    p = 0.017453292519943295
    hav = 0.5 - np.cos((lat2-lat1)*p)/2 + np.cos(lat1*p)*np.cos(lat2*p) * (1-np.cos((lon2-lon1)*p)) / 2
    dist = 12742 * np.arcsin(np.sqrt(hav))
    rav_inx = np.nanargmin(dist)
    unrav_inx = np.unravel_index(rav_inx, lon1.shape)
    return unrav_inx

In [64]:
df = pd.read_excel('MSMDI_stations_unique.xlsx')

In [65]:
df = df.drop(columns='Unnamed: 138')

In [99]:
months = ['Jan', 'Feb',
          'Mar', 'Apr', 'May',
          'Jun', 'Jul', 'Aug',
          'Sep', 'Oct', 'Nov', 'Dec']

In [100]:
cols = ['chl_surf_' + m + '_mg/m3' for m in months]
cols += ['chl_0_10_' + m + '_mg/m3' for m in months]

In [123]:
# add chl columns
df2 = pd.DataFrame(columns=cols)
df = pd.concat([df,df2])
df['ID'] = df['ID'].astype(int)

In [124]:
# table with stations coordinates
st = df.groupby('ID').first()[['long', 'lat']]

In [104]:
# main dataset
ds = xr.open_mfdataset('bottom/martini800*_monmean.nc',).squeeze()
# dataset for specific variables
par0 = xr.open_dataset('light/martini800_v9ae_z01_PAR0_monmean.nc').squeeze()
salt_surf = xr.open_dataset('surface/martini800_v9ae_salt_s41_monmean.nc').squeeze()
chl_surf = xr.open_dataset('surface/martini800_v9ae_light_Chl_s41_monmean.nc').squeeze()
chl_0_10 = xr.open_dataset('depthav0_10m/martini800_v9ae_light_Chl_0_10m_av_monmean.nc').squeeze()

In [105]:
chl_0_10

<xarray.Dataset>
Dimensions:          (ocean_time: 36, bnds: 2, eta_rho: 426, xi_rho: 480)
Coordinates:
  * ocean_time       (ocean_time) datetime64[ns] 2017-01-15 ... 2019-12-15
Dimensions without coordinates: bnds, eta_rho, xi_rho
Data variables:
    ocean_time_bnds  (ocean_time, bnds) datetime64[ns] ...
    lat_rho          (eta_rho, xi_rho) float64 ...
    light_Chl        (ocean_time, eta_rho, xi_rho) float32 ...
    lon_rho          (eta_rho, xi_rho) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.10 (https://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Tue Nov 29 16:01:52 2022: cdo -z zip_5 monmean martini800_v...
    NCO:          netCDF Operators version 5.0.3 (Homepage = http://nco.sf.ne...
    frequency:    mon
    CDO:          Climate Data Operators version 1.9.10 (https://mpimet.mpg.d...

In [106]:
lon_rho = ds.lon_rho.values
lat_rho = ds.lat_rho.values
# to exclude land from distance calculations
lon_rho[np.isnan(ds['DIN'].values[0,:,:])] = np.nan
lat_rho[np.isnan(ds['DIN'].values[0,:,:])] = np.nan

In [107]:
outnames = ['DIN', 'N4_n',
             'TotN', 'N1_p', 'TotP',
             'B1_DOC', 'light_POC',
             'salt', 'salt_surf',
             'temp', 'z01_PAR0',
             'chl_surf', 'chl_0_10']

In [108]:
variables = ['DIN', 'N4_n',
             'TotN', 'N1_p', 'TotP',
             'B1_DOC', 'light_POC',
             'salt', 'temp']
nv = len(variables)

In [109]:
# dictionary of variable names and lists of datasets
# {'vname': [st0, st1, st2 ... st55]}
vdict = dict((v, []) for v in outnames)

In [110]:
for i, row in st.iterrows():
    # index of closest MARTINI point to the station
    xloc, yloc = closest_inx(lat_rho, lon_rho, row['lat'], row['long'])
    
    # variables from main DS
    for v in variables:
        v3yr = ds[v].values[:,xloc,yloc].reshape(3,-1)
        vdict[v].append(v3yr)
    
    # salt_surf
    v3yr = salt_surf['salt'].values[:,xloc,yloc].reshape(3,-1)
    vdict['salt_surf'].append(v3yr)
    
    # z01_PAR0
    v3yr = par0['z01_PAR0'].values[:,xloc,yloc].reshape(3,-1)
    vdict['z01_PAR0'].append(v3yr)
    
    # Chl_surf
    v3yr = chl_surf['light_Chl'].values[:,xloc,yloc].reshape(3,-1)
    vdict['chl_surf'].append(v3yr)
    
    # Chl_0_10
    v3yr = chl_0_10['light_Chl'].values[:,xloc,yloc].reshape(3,-1)
    vdict['chl_0_10'].append(v3yr)

In [119]:
st

,long,lat
ID,,
0.0,8.830000,58.390000
1.0,7.602960,58.010900
2.0,8.198000,58.096100
3.0,8.066250,58.132300
4.0,8.035930,58.144700
5.0,8.391430,58.218800
6.0,8.361030,58.230520
7.0,8.380330,58.239550
8.0,8.428900,58.238200


In [125]:
for iv, outname in enumerate(outnames):  # martini variables
    for stinx in st.index:  # station index
        ist = df.index[df['ID'] == stinx].tolist()  # indexes of station [0:3], [3:6], [6:9] ...
        # fill the columns and rows in dataframe
        # ist - rows
        # 6+12*iv:6+12*(iv+1) - columns, variables for 12 months
        df.iloc[ist, 6+12*iv:6+12*(iv+1)] = vdict[outname][stinx]

In [126]:
df.to_excel('out_martini_chl.xlsx')

In [127]:
df

,stat_code,name,ID,long,lat,year,DIN_Jan [mmol N/m^3],DIN_Feb [mmol N/m^3],DIN_Mar [mmol N/m^3],DIN_Apr [mmol N/m^3],...,chl_0_10_Mar_mg/m3,chl_0_10_Apr_mg/m3,chl_0_10_May_mg/m3,chl_0_10_Jun_mg/m3,chl_0_10_Jul_mg/m3,chl_0_10_Aug_mg/m3,chl_0_10_Sep_mg/m3,chl_0_10_Oct_mg/m3,chl_0_10_Nov_mg/m3,chl_0_10_Dec_mg/m3
0,NaN,Arendal,0,8.830000,58.390000,2017.0,8.979590,11.547657,11.461661,11.018097,...,3.885201,2.766257,1.679893,1.266791,0.858215,0.877753,1.02968,0.992454,0.779359,0.479767
1,NaN,Arendal,0,8.830000,58.390000,2018.0,10.939911,11.236777,13.416727,13.538981,...,5.85179,2.868029,1.446093,1.138117,0.658601,0.690462,0.766069,1.102677,1.456852,1.20037
2,NaN,Arendal,0,8.830000,58.390000,2019.0,10.400820,10.818946,9.838261,11.544834,...,3.662263,2.544118,1.314517,1.031135,0.85074,0.772739,1.06018,1.159464,1.278286,0.63511
3,510,Eigebrekk,1,7.602960,58.010900,2017.0,8.039273,8.497822,8.969562,9.229198,...,2.47398,1.762152,1.687647,1.262237,1.050558,0.951188,0.908563,0.620501,0.427088,0.304342
4,510,Eigebrekk,1,7.602960,58.010900,2018.0,9.517045,8.113119,6.147118,7.744576,...,5.345798,3.291965,1.643052,2.106095,1.31343,0.856155,0.569643,0.823923,0.927551,1.004024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,NaN,"Håøyfjorden, BN1",54,10.556618,59.677308,2018.0,13.554756,12.221051,12.508015,13.573027,...,6.240458,2.410136,1.261876,0.690378,0.532707,0.349847,0.617521,1.76609,1.990951,1.606455
164,NaN,"Håøyfjorden, BN1",54,10.556618,59.677308,2019.0,11.022715,11.079479,12.055616,12.117801,...,3.14902,2.770841,1.637952,0.922955,0.605778,0.426564,0.785238,1.133151,1.207857,0.80331
165,0101020601-C,0101020601-C,55,10.549133,59.685789,2017.0,10.648481,13.890250,14.840036,14.541985,...,4.085019,1.870587,1.815057,0.851659,0.614158,0.445726,0.650136,1.137401,2.183577,1.211509
166,0101020601-C,0101020601-C,55,10.549133,59.685789,2018.0,13.744143,12.146265,12.459238,13.591425,...,6.278624,2.410203,1.25116,0.682295,0.530488,0.347866,0.613232,1.759301,2.0156,1.578661


In [111]:
closest_inx(lat_rho, lon_rho, 58.39, 8.83)

(272, 222)

In [112]:
lon_rho[272, 222], lat_rho[272, 222]

(8.835768031934263, 58.38998039410582)

In [115]:
ds['DIN'].isel(eta_rho=272, xi_rho=222).values

array([ 8.97959  , 11.547657 , 11.461661 , 11.018097 , 12.62449  ,
       10.660494 , 12.03216  , 10.564056 , 10.608728 ,  8.67279  ,
        8.3729105,  9.413659 , 10.939911 , 11.236777 , 13.416727 ,
       13.538981 , 13.157536 , 13.767018 , 14.03351  , 13.3628025,
       10.483186 ,  9.284698 , 10.717784 , 10.182386 , 10.40082  ,
       10.818946 ,  9.838261 , 11.544834 , 12.195189 , 12.265044 ,
       12.528326 , 11.197827 , 10.488713 ,  9.5013075, 12.096882 ,
        9.943212 ], dtype=float32)